# Training LSTM to generate Cyberpunk stories
### Cus I need to pretend I've done NLP

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
%cd gdrive/My\ Drive/ML/textRNN

/content/gdrive/My Drive/ML/textRNN


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import glob
import re

In [4]:
neuromancer = open("neuromancer.txt").read()

chapter_re = "\\n\\n\\n\\n\d+ \\n\\n\\n"
match = re.findall(chapter_re, neuromancer)
match

['\n\n\n\n1 \n\n\n',
 '\n\n\n\n2 \n\n\n',
 '\n\n\n\n3 \n\n\n',
 '\n\n\n\n4 \n\n\n',
 '\n\n\n\n5 \n\n\n',
 '\n\n\n\n6 \n\n\n',
 '\n\n\n\n7 \n\n\n',
 '\n\n\n\n8 \n\n\n',
 '\n\n\n\n9 \n\n\n',
 '\n\n\n\n10 \n\n\n',
 '\n\n\n\n11 \n\n\n',
 '\n\n\n\n12 \n\n\n',
 '\n\n\n\n13 \n\n\n',
 '\n\n\n\n14 \n\n\n',
 '\n\n\n\n15 \n\n\n',
 '\n\n\n\n16 \n\n\n',
 '\n\n\n\n17 \n\n\n',
 '\n\n\n\n18 \n\n\n',
 '\n\n\n\n19 \n\n\n',
 '\n\n\n\n20 \n\n\n',
 '\n\n\n\n21 \n\n\n',
 '\n\n\n\n22 \n\n\n',
 '\n\n\n\n23 \n\n\n',
 '\n\n\n\n24 \n\n\n']

In [5]:
chapters = re.split(chapter_re, neuromancer)

# for chapter in chapters:
#     print(chapter[:100])
#     print("***********************")

In [6]:
chapters_ = []
for chapter in chapters:
    if len(chapter) > 100:
        chapter = re.sub("( \\n)(?=[a-zA-Z])", " ", chapter)
        chapters_.append(chapter)
        
for chapter in chapters_[:5]:
    print(chapter[:100])
    print("***********************")
    
chapters = chapters_

The sky above the port was the color of television, tuned to a dead channel. 

"It’s not like I’m us
***********************
After a year of coffins, the room on the twenty-fifth floor of the Chiba Hilton seemed enormous. It 
***********************
Home. 

Home was BAMA, the Sprawl, the Boston-Atlanta Metropolitan Axis. 

Program a map to display 
***********************
Case sat in the loft with the dermatrodes strapped across his forehead, watching motes dance in the 
***********************
The medical team Molly employed occupied two floors of an anonymous condo-rack near the old hub of B
***********************


In [7]:
chapters[0]

'The sky above the port was the color of television, tuned to a dead channel. \n\n"It’s not like I’m using," Case heard someone say, as he shouldered his way through the crowd around the door of the Chat. "It’s like my body’s developed this massive drug deficiency." It was a Sprawl voice and a Sprawl joke. The Chatsubo was a bar for professional expatriates; you could drink there for a week and never hear two words in Japanese. Ratz was tending bar, his prosthetic arm jerking monotonously as he filled a tray of glasses with draft Kirin. He saw Case and smiled, his teeth a web work of East European steel and brown decay. Case found a place at the bar, between the unlikely tan on one of Lonny Zone’s whores and the crisp naval uniform of a tall African whose cheekbones were ridged with precise rows of tribal scars. "Wage was in here early, with two Joe boys," Ratz said, shoving a draft across the bar with his good hand. "Maybe some business with you, Case?" Case shrugged. The girl to his 

In [8]:
mona_lisa_overdrive = open("mona_lisa_overdrive.txt").read()

chapter_re = "\\n\d+\\n"
match = re.findall(chapter_re, mona_lisa_overdrive)
match

[]

In [9]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(dtype = np.int32)
encoder.fit(np.array(list(neuromancer)).reshape(-1,1))
print(encoder.categories_)

num_chars = len(encoder.categories_[0])

[array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0',
       '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A',
       'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—',
       '’'], dtype='<U1')]


In [10]:
chapters_encoded = [encoder.transform(np.array(list(c)).reshape(-1,1)) for c in chapters]

In [11]:
dataset = tf.data.Dataset.from_generator(lambda: iter(chapters_encoded), chapters_encoded[0].dtype)

In [12]:
dataset = tf.data.Dataset.from_generator(lambda: iter(chapters_encoded), chapters_encoded[0].dtype)
# dataset = dataset.map(lambda chapter: tf.reshape(chapter, (-1,1)))

# print(list(dataset.take(1))[0].shape) #One chapter instance

#Now convert each chapter to windows and flatten so just loads of windows
window_length = 200
batch_size = 32

def chapter_to_windows(c):
    ds = tf.data.Dataset.from_tensor_slices(c)
    ds = ds.window(window_length+1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window: window.batch(window_length+1))
    return ds

dataset = dataset.map(chapter_to_windows)
dataset = dataset.flat_map(lambda ds: ds) #Fuck knows

# print(list(dataset.take(2))[0][:10])


#Now batch, convert into shifted etc.
dataset = dataset.shuffle(10000)
dataset = dataset.batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))

# print(list(dataset.take(1))[0])



#Then one hot LAST (I guess)

def one_hotter(X_batch, Y_batch):
    X = tf.reshape(tf.one_hot(X_batch, depth = num_chars, dtype = np.int32)[:,:,0,:],(-1,window_length,num_chars)) #lol
    Y = tf.reshape(Y_batch, (-1, window_length))[:,:,None] #lol
    return (X,Y)

dataset = dataset.map(one_hotter)
dataset = dataset.prefetch(2)

#Don't one-hot the Y because _sparse_ categorical loss sorted it out for you

In [16]:
np.array(list(dataset.take(1))[0]).shape

(2, 32, 200)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences = True, input_shape = [None, num_chars],
                         dropout = 0.05),
    tf.keras.layers.LSTM(128, return_sequences = True,
                         dropout = 0.05),
    tf.keras.layers.Dense(num_chars)
])

# model = tf.keras.models.Sequential([
#     tf.keras.layers.LSTM(256, return_sequences = True, input_shape = [None, num_chars]),
#     tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_chars))
# ])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

optimizer = tf.keras.optimizers.Adam(lr = 0.01)

model.compile(loss = loss, optimizer = optimizer)

In [ ]:
filepath="models/neuromancer-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, mode='min')

history = model.fit(dataset, epochs = 5,  callbacks=[checkpoint])

Epoch 1/5
    264/Unknown - 21s 81ms/step - loss: 2.4148

KeyboardInterrupt: ignored

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, None, 128)         106496    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dense_1 (Dense)              (None, None, 79)          10191     
Total params: 248,271
Trainable params: 248,271
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.load_weights("models/neuromancer_mk1.hdf5")

In [ ]:
num_chars

1

In [ ]:
vocab = encoder.categories_[0]
num_stories = 1
num_chars_max = 1000
stories = []

for i in range(num_stories):
  init_char = np.random.choice(list("qwertyuiopasdfghjklzxcvbnm"))
  text = init_char.upper()
  text = "Th"

  n = 0
  while n < num_chars_max:

    text_ids = encoder.transform(np.array(list(text)).reshape(-1,1))
    text_onehot = tf.one_hot(text_ids, depth = num_chars, dtype = np.int32)[:,0,:]

    out = model(text_onehot[None,:,:])[0,-1]
    out = np.array(out)

    if text[-1] == " ":
      temp = 1.
    else:
      temp = 0.3

    out /= temp
    out = out - out.max()

    softmaxed = np.exp(out)
    softmaxed /= softmaxed.sum()
    # indices = np.argsort(softmaxed)[::-1][:5]

    chosen_index = np.random.choice(np.arange(num_chars), p = softmaxed) 
    chosen_char = encoder.categories_[0][chosen_index]
    n+= 1
    text += chosen_char
    if chosen_char == "¶": break
    
  stories.append(text)


  textfile = open(f'output/neuromancer_story{i}.txt', 'w')
  textfile.write(text)
  textfile.close()

  print("************************")
  print(f"{text}")


************************
The thing he could feel it there. 

"Where is orgas. I know how that do you open their . . ." She smiled. She was jest construct something the  it couldn’t have the Emergency crowd while he made Liesk and collarbones display be the bed and found curve, not already of something their number striped sense of silence. The faces around them to squired up. Nothing he remembered termical on the condos that come back between them, a markets of his skull with x-rays of silks. Wintermute is Panacher." 

"I never screw up." 

"What he liken?" Case was stenciled in the gold and formed himself at all the window, the streetlight managed to allow at a cloud who gradually the blades of flosts had been his little friggs lit a few deck, a knocian revealing it about them, he was gone, beyond the register. When he could see the drug dirension, that had had to remember a good place for meat know when he showed ever had come of Desiderata, fumbling to some kind of an explosion. His

In [ ]:
for n, story in enumerate(stories):
  textfile = open(f'output/neuromancer_story{n}.txt', 'w')
  textfile.write(story)
  textfile.close()

In [ ]:
d = list(dataset.take(1))[0]

In [ ]:
vocab[np.argmax(d[0][0], axis = 1)]

array(['c', 'o', 'r', 'p', 'o', 'r', 'a', 't', 'e', 'r', 'c', 'o', 'l',
       'o', 'g', 'i', 'e', 's', '.', ' '], dtype='<U1')

In [ ]:
vocab[d[1][0]]

array(['s', ' ', 'l', 'i', 'k', 'e', ' ', 'a', ' ', 'd', 'e', 'r', 'a',
       'n', 'g', 'e', 'd', 'x', 'p', 'e', 'r', 'i', 'm', 'e', 'n', 't',
       ' ', 'i', 'n', ' ', 's', 'o', 'c', 'i', 'a', 'l', ' ', 'D', 'a',
       'r', 'w', 'i', 'n', 'i', 's', 'm', ',', ' ', 'd', 'e', 's', 'i',
       'g', 'n', 'e', 'd', ' ', 'b', 'y', ' ', 'a', ' ', 'b', 'o', 'r',
       'e', 'd', ' ', 'r', 'e', 's', 'e', 'a', 'r', 'c', 'h', 'e', 'r',
       ' ', 'w', 'h', 'o', ' ', 'k', 'e', 'p', 't', ' ', 'o', 'n', 'e',
       ' ', 't', 'h', 'u', 'm', 'b', ' ', 'p', 'e'], dtype='<U1')

In [ ]:
print([np.argwhere(vocab == c) for c in 'The'])

[array([[44]]), array([[58]]), array([[55]])]


In [ ]:
np.argwhere(vocab == 'T')
T = np.full((2,num_chars), 0.)
T[0,44] = 1.
T[1,58] = 1.

In [ ]:
logits = model(T[None,:,:])
probs = np.exp(logits)
probs /= probs.sum()

In [ ]:
np.argmax(probs[0,1])

55

In [ ]:
probs.shape

(79,)

In [ ]:
vocab

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0',
       '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A',
       'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—',
       '’'], dtype='<U1')

In [ ]:
data